<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# EODHistoricalData API Wrapper

**`tpqeod` Tutorial**

Dr. Yves J. Hilpisch | The Python Quants GmbH | https://tpq.io

This notebook provides a comprehensive tutorial on using the `tpqeod` Python wrapper class for accessing financial data from EODHistoricalData.com. It covers various functionalities, from retrieving historical data and fundamentals to working with real-time data streams.

**Note**: This wrapper is currently a work in progress, and significant changes may occur in future versions.

## Introduction to `tpqeod`

The `tpqeod` class is a convenient Python wrapper designed to simplify interactions with the EODHistoricalData API. To utilize its capabilities, you will need an active account with [EODHistoricalData](https://bit.ly/eod_data) and a generated API key.

Before proceeding, ensure you have your API key ready. A common practice is to store it in a separate Python file, for example, `creds.py`, as shown below:

```python
eod_key = "YOUR_EOD_KEY"
```

This allows you to keep your credentials secure and separate from your main codebase.

### Importing the Wrapper and Initializing the API

First, we import the `tpqeod` class and then load our API key. Once the API key is loaded, we can initialize the `tpqeod` object, which will be used for all subsequent API calls.

In [ ]:
!git clone https://github.com/tpq-classes/data_science_basics.git
import sys
sys.path.append('data_science_basics')


In [ ]:
from tpqeod import tpqeod
import pandas as pd
import datetime as dt
from pylab import plt
import warnings as w; w.simplefilter('ignore') # Ignore warnings

plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format='svg'

In [ ]:
# In a real scenario, you would have your API key in a file like 'creds.py'
# For demonstration, we'll define a placeholder. REPLACE WITH YOUR ACTUAL KEY.
# eod_key = "YOUR_EOD_KEY"
%run creds.py

# Initialize the API object
api = tpqeod(eod_key)

## Historical End-of-Day (EOD) Data

The `get_eod_data()` method allows you to retrieve historical End-of-Day (EOD) data for a given symbol. You can specify the exchange, period (daily, weekly, monthly), start and stop dates, and the order of the data.

### Getting Adjusted Close Prices for Multiple Stocks

This example demonstrates how to fetch the 'Adjusted_close' price for a list of popular tech stocks and store them in a pandas DataFrame.

In [ ]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'NFLX', 'META', 'GOOG', 'NVDA']
prices = pd.DataFrame()
for ticker in tickers:
    print(f"Fetching EOD data for {ticker}...")
    # Fetch daily EOD data from May 3, 2015 onwards
    d = api.get_eod_data(ticker, start=pd.Timestamp('2015-05-03'))
    prices[ticker] = d['Adjusted_close']
print("\nDone fetching prices.")

Let's inspect the `prices` DataFrame to see its structure and a sample of the data.

In [ ]:
print("DataFrame Info:")
prices.info()

print("\nLast 7 rows for selected columns (AMZN, NFLX, META, GOOG):")
prices.iloc[-7:, 3:7]

### Visualizing Normalized Prices

To compare the performance of different stocks, it's often useful to normalize their prices to a starting point. Here, we normalize the first three stocks (AAPL, MSFT, AMZN) to their initial price and plot their trajectories.

In [ ]:
print("Plotting normalized prices for AAPL, MSFT, AMZN...")
(prices.iloc[:, :3] / prices.iloc[0, :3]).plot(figsize=(10, 6), title='Normalized Stock Prices (AAPL, MSFT, AMZN)');
plt.xlabel("Date")
plt.ylabel("Normalized Price")
plt.grid(True)

## Fundamental Data

The `tpqeod` wrapper provides access to comprehensive fundamental data for various financial instruments. The `get_fundamentals()` method is your gateway to this information.

### Getting Fundamentals for Multiple Stocks

We'll retrieve a summary of 'Highlights' fundamental data for our list of tech tickers. The `as_json=True` parameter can be used to get the raw JSON response, while omitting it will return a `fundamentals` object that provides structured access to different tables of fundamental data.

In [ ]:
fundamentals_summary = pd.DataFrame()
fundamentals_full_data = {}
for ticker in tickers:
    print(f"Fetching fundamentals for {ticker}...")
    # Fetch fundamental data and store the 'Highlights' section
    f_full = api.get_fundamentals(ticker, as_json=True)
    fundamentals_summary = pd.concat((fundamentals_summary, 
                                     pd.DataFrame(f_full['Highlights'], index=[ticker])))
    fundamentals_full_data[ticker] = f_full # Store full data for later inspection
print("\nDone fetching fundamentals.")

Let's look at the 'Highlights' section for Microsoft (MSFT) from the full fundamental data, and a transposed view of the `fundamentals_summary` for selected companies.

In [ ]:
print("MSFT Highlights from full fundamental data:")
print(fundamentals_full_data['MSFT']['Highlights'])

print("\nTransposed fundamentals summary for selected companies (NFLX, META, GOOG):")
fundamentals_summary.T.iloc[:, 3:6]

In [ ]:
fundamentals_full_data['MSFT']['General']

In [ ]:
# import json

In [ ]:
# with open('fundamentals.json', 'w') as f:
#    json.dump(fundamentals_full_data['MSFT'], f, indent=4)

### Fundamentals for a Complete Index

You can also retrieve components of an index and then iterate through them to fetch their individual fundamental data. Here, we get the components of the DAX index (GDAXI) and then fetch their 'Highlights' fundamental data.

In [ ]:
print("Fetching DAX constituents...")
dax_constituents = api.get_fundamentals(symbol='GDAXI', exchange='INDX', table='Components')
print("\nDAX Constituents Head:")
dax_constituents.head()

In [ ]:
%%time

print("Fetching highlights for DAX constituents (this may take a while)...")

highlights_dax = list()
for ticker_code in dax_constituents['Code']:
    print(f"Fetching {ticker_code}... ", end='')
    # For DAX constituents, the exchange is typically 'XETRA'
    highlights_dax.append(api.get_fundamentals(symbol=ticker_code,
                                           exchange='XETRA',
                                           table='Highlights'))

dax_fundamentals = pd.DataFrame(highlights_dax, index=dax_constituents['Code'])
print("\nDone fetching DAX fundamentals.")

Reviewing a subset of the collected DAX fundamentals:

In [ ]:
# api.get_fundamentals??

In [ ]:
print("Transposed DAX fundamentals for selected companies:")
dax_fundamentals.T.iloc[:, 5:10]

## Exchange Information

The `tpqeod` wrapper allows you to retrieve information about available exchanges and their associated tickers.

### Available Exchanges

You can get a list of all supported exchange codes using `get_exchange_codes()`.

In [ ]:
print("Sorted list of available exchange codes:")
print(sorted(api.get_exchange_codes()))

### Fundamentals for Specific Exchanges

The `get_exchange_data()` method provides detailed information about a specific exchange. You can also pass 'All' to get a DataFrame of all exchanges.

In [ ]:
print("Data for ZSE (Zimbabwe Stock Exchange):")
print(api.get_exchange_data('ZSE'))

print("\nData for BE (Berlin Stock Exchange):")
print(api.get_exchange_data('BE'))

print("\nHead of data for all exchanges:")
print(api.get_exchange_data('All').head())

### Available Tickers for an Exchange

To get a list of all tickers available on a specific exchange, use `get_ticker_list()`.

In [ ]:
print("Head of ticker list for ZSE (Zimbabwe Stock Exchange):")
api.get_ticker_list('ZSE').head()

## Searching for Symbols

The `search()` method allows you to search for symbols based on a query string, with options to limit results and filter by asset type.

In [ ]:
print("Searching for 'tesla' with a limit of 25 and asset type 'all':")
api.search('tesla', limit=25, asset_type='all')

## Economic Calendar

The `get_calendar()` method provides access to economic events such as earnings, IPOs, and splits. You can filter by event type, symbol, and date range.

The following event types are available:
 - `earnings`
 - `ipos`
 - `splits`

Data can be restricted to a single ticker by providing the `symbol` parameter. If `start` or `stop` dates are omitted, the present day is used as the start date, and the present day plus 7 days is used as the stop date.

In [ ]:
print("Economic calendar for splits in October 2023:")
api.get_calendar('splits', start=dt.datetime(2023, 10, 1), stop=dt.datetime(2023, 10, 31))

## Historical Splits and Dividends

You can retrieve historical stock splits and dividend payments for a given symbol using `get_hist_splits()` and `get_hist_dividends()` respectively.

In [ ]:
print("Historical splits for AAPL since January 1, 2000:")
api.get_hist_splits('AAPL', exchange='US', start=dt.datetime(2000, 1, 1))

In [ ]:
print("Historical dividends for AAPL (US exchange):")
api.get_hist_dividends('AAPL', exchange='US')

## Historical Tick Data

The `get_tick_data()` method allows you to retrieve historical tick-level data. This is typically available only for 'US' exchanges and provides granular trade information.

In [ ]:
%%time
print("Fetching tick data for AAPL on May 23, 2025...")
ticks = api.get_tick_data('AAPL', start=dt.datetime(2025, 5, 23, 0, 0, 0),
                         stop=dt.datetime(2025, 5, 23, 23, 59, 59))
print("\nDone fetching tick data.")

Inspect the structure and head of the tick data DataFrame:

In [ ]:
print("Tick data info:")
ticks.info()

print("\nTick data head:")
ticks.head()

## Technical Indicators

The `get_technical_indicator()` method allows you to calculate various technical indicators for a given symbol and exchange. You can specify the indicator function, period, and date range.

Available values for the indicator function are:
 - `'splitadjusted'`
 - `'avgvol'`
 - `'avgvolccy'`
 - `'sma'` (Simple Moving Average)
 - `'ema'` (Exponential Moving Average)
 - `'wma'` (Weighted Moving Average)
 - `'volatility'`
 - `'rsi'` (Relative Strength Index)
 - `'stddev'` (Standard Deviation)
 - `'stochastic'`
 - `'stochrsi'`
 - `'slope'`
 - `'dmi'`
 - `'adx'`
 - `'macd'` (Moving Average Convergence Divergence)
 - `'atr'` (Average True Range)
 - `'cci'` (Commodity Channel Index)
 - `'sar'` (Stop And Reverse)
 - `'bbands'` (Bollinger Bands)

Refer to the [EODHistoricalData Technical Indicators API documentation](https://eodhd.com/financial-apis/technical-indicators-api/) for more details on each indicator and their parameters.

### Example: Simple Moving Average (SMA)

Here, we'll calculate a 7-period Simple Moving Average (SMA) for Amazon (AMZN) and compare it with its adjusted close price.

In [ ]:
print("Calculating 7-period SMA for AMZN...")
mavg = api.get_technical_indicator('AMZN', 'US', function='sma', period=7,
                                   start=dt.datetime(2024, 1, 1),
                                   stop=dt.datetime(2024, 12, 31))
print("SMA head:")
mavg.head()

In [ ]:
print("Fetching Adjusted Close prices for AMZN...")
prices_amzn = api.get_eod_data('AMZN', 'US', start=dt.datetime(2024, 1, 11),
                        stop=dt.datetime(2024, 12, 31))['Adjusted_close']
print("AMZN prices head:")
prices_amzn.head()

In [ ]:
data_combined

In [ ]:
print("Combining prices and SMA and plotting normalized values...")
data_combined = pd.concat([prices_amzn, mavg], axis=1).dropna()
(data_combined / data_combined.iloc[0]).plot(figsize=(10, 6), title='Normalized AMZN Price vs. 7-Period SMA');
plt.xlabel("Date")
plt.ylabel("Normalized Value")
plt.grid(True)

## Live Data Streams

`tpqeod` also provides functionality to stream live data for cryptocurrencies, forex, trades, and quotes. The data is accumulated in internal attributes of the `api` object (e.g., `api.crypto_data`).

**Important**: These streaming functions initiate separate threads for data reception. To stop streaming, you must explicitly call the corresponding `close_` method (e.g., `close_crypto_stream()`).

### Live Crypto Data (stream)

The `get_live_crypto_data()` method starts streaming real-time cryptocurrency data. The received data is stored in the `api.crypto_data` dictionary, with symbols as keys and pandas DataFrames as values.

In [ ]:
import time

print("Starting live crypto data stream for ETH-USD...")
api.get_live_crypto_data('ETH-USD')

After a short delay, you can inspect the collected data:

In [ ]:
time.sleep(5)  # Wait for some data to accumulate
print("Collected crypto data for ETH-USD:")
api.crypto_data

You can subscribe to multiple symbols simultaneously.

In [ ]:
print("Starting live crypto data stream for BTC-USD...")
api.get_live_crypto_data('BTC-USD')

In [ ]:
time.sleep(5)  # Wait for more data
print("Collected crypto data for both ETH-USD and BTC-USD:")
api.crypto_data

You can access specific symbol data from the `crypto_data` dictionary:

In [ ]:
print("BTC-USD specific data:")
api.crypto_data['BTC-USD']

**Always remember to close the stream when you are done to release resources.**

In [ ]:
print("Closing crypto stream...")
api.close_crypto_stream()

### Other Live Data Streams

Similar to cryptocurrency data, `tpqeod` supports other live data streams:
 - **Forex Data**: `.get_live_forex_data(symbol)` (Data stored in `api.forex_data`)
 - **Trade Data**: `.get_live_trade_data(symbol)` (Data stored in `api.trade_data`)
 - **Quote Data**: `.get_live_quote_data(symbol)` (Data stored in `api.quote_data`)

Remember to use their respective `close_` methods (e.g., `api.close_forex_stream()`, `api.close_trade_stream()`, `api.close_quote_stream()`) to stop the streams.

<img src='https://certificate.tpq.io/tpq_logo.png' width="35%" align="right">

<br><br><a href="https://tpq.io" target="_blank">http://tpq.io</a> | <a href="https://linktr.ee/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>